# Optimización de la Trayectoria de Ascenso de una Aeronave A320 utilizando Evolución Diferencial

#Marco A. Erazo. Dir: Leonardo A. Pachon

Este documento describe el código Python que implementa un algoritmo de Evolución Diferencial para optimizar la trayectoria de ascenso de una aeronave A320. El objetivo es encontrar la combinación óptima de velocidades y ángulos de ascenso en cada punto discretizado de la trayectoria que minimice la función de costo, que considera el consumo de combustible y el tiempo de vuelo, tal como se define en el problema original de Airbus Quantum Computing Challenge.

El código utiliza la biblioteca SciPy para implementar el algoritmo de Evolución Diferencial y define la función de costo y las restricciones del problema, basándose en las ecuaciones proporcionadas en el enunciado del problema. Además, se incluye un mecanismo de perturbación para explorar el espacio de soluciones de forma más exhaustiva y escapar de mínimos locales.

**Descripción del Código**

El código se divide en varias secciones:

1. Definición de constantes y parámetros del problema.
2. Definición de funciones auxiliares para calcular variables aerodinámicas y de la atmósfera.
3. Definición de la función objetivo (función de costo).
4. Implementación del algoritmo de Evolución Diferencial.

**Definición de Constantes y Parámetros**

- Se definen las constantes del problema, como los coeficientes aerodinámicos, la eficiencia del combustible, la altitud inicial y final, la masa inicial, la velocidad inicial, etc.
- Se definen las unidades y se realizan las conversiones necesarias para mantener la consistencia dimensional.
- Se definen los límites de las variables de optimización (velocidad y ángulo de ascenso).

In [1]:
import numpy as np
from math import sin, tan, sqrt, pow, asin, atanh, log, exp, pi, nan
from scipy.optimize import differential_evolution
import time

Cx_0 = 0.014
k = 0.09
Cz_max = 0.7
S_REF = 120
η = 0.06/3600
Zp_I = 10000 * 0.3048
Zp_F = 36000 * 0.3048

π = pi

m_I = 60000
CAS_I = 250*0.5144444444444445
VMO = 350*0.5144444444444445
MMO = 0.82
M_CRZ = 0.80
L = 400000
s_F=L

Vz_min = 1.52400
g_0 = 9.80665
CI = 30/60

m_0 = m_I
t_0 = 0
s_0 = 0
λ_0 = 1

Ts_0 = 288.15
ρ_0 = 1.225
L_z = -0.0065
g_0 = 9.80665
R = 287.05287
α_0 = -g_0/R/L_z

**Definición de Funciones Auxiliares**

- Se definen funciones para calcular la altitud `Zp(i)`, el empuje máximo `F_N_MCL(i)`, la densidad del aire `ρ(i)`, el número de Mach `M(l)`, la velocidad calibrada del aire `CAS(l)` y la velocidad aerodinámica real `TAS_I`.
- Estas funciones se utilizan para calcular la función objetivo y las restricciones del problema.

In [4]:
def F(x):
    array_ejemplo = np.array(x)
    x1, x2 = np.array_split(array_ejemplo, 2)
    N=len (x1)+1
    x1 = np.array(x1)
    x2 = np.array(x2)

    def Zp(i):
        return Zp_I + i*(Zp_F - Zp_I)/(N)

    def F_N_MCL(i):
        return 140000 - 2.53*Zp(i)/0.3048

    def ρ(i):
        return ρ_0 * ((Ts_0 + L_z*Zp(i))/Ts_0)**(α_0 - 1)

    def M(l):
        return v[l]/sqrt(1.4*R*((Ts_0) + L_z*Zp(l)))

    def CAS(l):
        arg = (7*R*Ts_0) * (((Ts_0/(Ts_0 + L_z*Zp(l)))**-α_0 *
                                 (pow((1 + (v[l]**2/(7*R*(Ts_0 + L_z*Zp(l))))),3.5) - 1) + 1)**(1/3.5) - 1)
        if arg < 0:
            return nan
        else:
            return sqrt(arg)

    TAS_I = sqrt    (7*R*(Ts_0 + L_z*Zp_I) * ((((Ts_0 + L_z*Zp_I)/Ts_0)**-α_0 * ((1 + CAS_I**2/(7*R*Ts_0))**3.5 - 1) + 1)**(1/3.5) - 1))

    v_0 = TAS_I
    Cz_0 = m_0*g_0/(0.5*ρ(0)*v_0**2*S_REF)
    γ_0 = asin  ((F_N_MCL(0)-0.5*ρ(0)*v_0**2*S_REF*(Cx_0 + k*Cz_0))/(m_0*g_0))
    ρ_F = ρ_0 * ((Ts_0 + L_z*Zp_F)/Ts_0)**(α_0-1)
    v_F = M_CRZ * sqrt(1.4*R*(Ts_0 + L_z*Zp_F))

    def Cz_ip(i):
        return (2 * m[i+1] * sin(γ[i+1]) * ((2 * γ[i+1] - 2 * γ[i]) / (Zp(i+1) - Zp(i)) - (ρ(i) * S_REF * Cz[i]) / (2 * m[i] * sin(γ[i])) \
                                            + g_0 / (v[i+1]**2 * tan(γ[i+1])) + g_0 / (v[i]**2 * tan(γ[i]))))/(ρ(i+1) * S_REF)

    def m_ip(i):

        A = (v[i+1]-v[i])/(Zp(i+1)-Zp(i))

        L = (-g_0/v[i+1] + (λ[i]*F_N_MCL(i))/(m[i]*v[i]*sin(γ[i]))
            - (0.5*ρ(i)*v[i]*S_REF*(Cx_0+k*Cz[i]**2))/(m[i]*sin(γ[i]))
            - g_0/v[i])

        H = (4*sin(γ[i+1])/(ρ(i+1)*S_REF)) * ((γ[i+1]-γ[i])/(Zp(i+1)-Zp(i)) +
            g_0/(2*v[i+1]**2*tan(γ[i+1])) - (ρ(i)*S_REF*Cz[i])/(4*m[i]*sin(γ[i])) +
            g_0/(2*v[i]**2*tan(γ[i])))

        I = (-2*v[i+1]*sin(γ[i+1])/(η*F_N_MCL(i+1))) * (1/(Zp(i+1)-Zp(i)))

        J = (2*v[i+1]*sin(γ[i+1])/(η*F_N_MCL(i+1))) * (m[i]/(Zp(i+1)-Zp(i))) - (v[i+1]*sin(γ[i+1])*λ[i]*F_N_MCL(i))/(F_N_MCL(i+1)*v[i]*sin(γ[i]))

        numerator_1 = (-2.0*A*v[i+1]*sin(γ[i+1]) + F_N_MCL(i+1)*I + L*v[i+1]*sin(γ[i+1]))
        numerator_2 = (2.0*sqrt(A**2*v[i+1]**2*sin(γ[i+1])**2 - A*F_N_MCL(i+1)*I*v[i+1]*sin(γ[i+1]) -
                                    A*L*v[i+1]**2*sin(γ[i+1])**2 - 0.25*Cx_0*H**2*S_REF**2*k*ρ(i+1)**2*v[i+1]**4 +
                                    0.25*F_N_MCL(i+1)**2*I**2 + 0.5*F_N_MCL(i+1)*H**2*J*S_REF*k*ρ(i+1)*v[i+1]**2 +
                                    0.5*F_N_MCL(i+1)*I*L*v[i+1]*sin(γ[i+1]) + 0.25*L**2*v[i+1]**2*sin(γ[i+1])**2))
        denominator = H**2*S_REF*k*ρ(i+1)*v[i+1]**2

        m_i_plus_1_positive = (numerator_1 + numerator_2)/denominator
        m_i_plus_1_negative = (numerator_1 - numerator_2)/denominator

        return m_i_plus_1_positive

    def s_ip(i):
        return s[i] + 0.5 * (Zp(i+1) / tan(γ[i+1]) + Zp(i+1) / tan(γ[i]) - Zp(i) / tan(γ[i+1]) - Zp(i) / tan(γ[i]))

    def t_ip(i):
        return t[i] + 0.5 * ((Zp(i+1) - Zp(i)) / (v[i+1] * sin(γ[i+1])) + (Zp(i+1) - Zp(i)) / (v[i] * sin(γ[i])))

    def λ_ip(i):
        term1 = -2 * (v[i+1] * sin(γ[i+1])) / (η * F_N_MCL(i+1))
        term2 = (m[i+1] - m[i]) / (Zp(i+1) - Zp(i))
        term3 = (v[i+1] * sin(γ[i+1]) * λ[i] * F_N_MCL(i)) / (F_N_MCL(i+1) * v[i] * sin(γ[i]))
        return (term1 * term2) - term3

    def Θ(N_,_𝑣,γ_):
        global N, v, γ, m, s, t, 𝜆, Cz, v_, P
        N=N_ ; P=True ; v=_𝑣; γ=γ_#; v_=v[N-1]
        m = [m_0]; s=[s_0]; t=[t_0]; λ=[λ_0]; Cz=[Cz_0]
        Cz_i = Cz_0
        for i in range (0, N-1, 1):
            if len(v) <= (i+1) or v[i+1]*sin(γ[i+1]) < Vz_min or CAS(i+1) > VMO : P=False; N = i+1; break
            m.append(m_ip(i))
            Cz.append(Cz_ip (i))
            λ.append(λ_ip(i))
            s.append(s_ip(i))
            t.append(t_ip(i))
            if (λ[i+1] > 1 or λ[i+1] < 0) or (Cz[i+1] > Cz_max or M(i+1) > MMO):
                P=False; break
        return


    def A():
        return (-ρ_F * S_REF * Cx_0) / (2 * m[N-1]) - (6 * k * m[N-1] * g_0**2) / (ρ_F * S_REF * v[N-1]**4)

    def B():
        return (16 * k * m[N-1] * g_0**2) / (ρ_F * S_REF * v[N-1]**3)

    def C():
        return (F_N_MCL(N-1) / m[N-1]) - (12 * k * m[N-1] * g_0**2) / (ρ_F * S_REF * v[N-1]**2)

    def D(A, B, C):
        return (B**2 - 4 * A * C)**0.5

    def t_B():
        global kuo45789
        kuo45789 = (2/D(A(), B(), C())) * (atanh((2*A()*v[N-1] + B())/D(A(), B(), C())) - atanh((2*A()*v_F + B())/D(A(), B(), C())))
        return t[N-1] + kuo45789


    def m_B():
        return m[N-1] - η * λ[N-1] * F_N_MCL(N-1) * (t_B() - t[N-1])

    def s_B():
        return s[N-1] + (1/A()) * log((D(A(), B(), C())-2*A()*v_F-B())/(D(A(), B(), C())-2*A()*v[N-1]-B())) - (B()+D(A(), B(), C()))/(2*A()) * (t_B() - t[N-1])

    def m_F():
        return m_B() * exp((-2 * η * g_0 * sqrt(k * Cx_0) / v_F) * (s_F - s_B()))

    def t_F():
        return t_B() + (s_F - s_B()) / v_F

    def φ(v,γ):
        Θ(N,v,γ)
        if P == True:
            nada = -m_F()  + CI*(t_B() - s_B()/v_F)
            dt = [j-i for i, j in zip(t[:-1], t[1:])]
            dtp = dt + [kuo45789]
            aaa = [aa *  bb for aa, bb in zip(v.tolist(), np.sin(γ).tolist())]
            ccc = [aa *  bb for aa, bb in zip(v.tolist(), np.cos(γ).tolist())]
            bbb = [aa_ *  bb_ for aa_, bb_ in zip(aaa, dtp)]
            ddd = [aa_ *  bb_ for aa_, bb_ in zip(ccc, dtp)]
            if sum(ddd) <= 350000:
                return sum(bbb)
            else:
                return nan
        else:
            return nan


    return φ(np.concatenate(([v_0], x1)), np.concatenate(([γ_0], x2*π/180)))

**Definición de la Función Objetivo (Función de Costo)**

- Se define la función `F(x)` y se prueba con un punto ($v-\gamma$) en particular, la función `F(x)` representa la función de costo a minimizar.


In [6]:
def objective_function(x):
    a = -F(x)
    if a > 0:
        return a
    else:
        return 88000

def FF(x):
    a = F(x)
    if a > 0:
        return a
    else:
        return 99999999999 # simplemente un valor alto

FF([136.72340757119528, 139.57664297548615, 151.95327777594713, 137.59477992202238, 144.6064094307896, 157.97701229735156, 165.27849090941032, 176.29415090821374, 186.22651828660727, 201.30525703185089, 215.95044899534938, 223.48183794549536, 210.68440649065496, 183.27145147267578, 191.76996308400928, 215.08825582934148, 207.09020190557808, 205.43367394272582, 206.13765224810723, 201.98441712711158, 198.12722666850672, 196.56099273807874, 201.18684708393573, 203.90763941834703, 208.15028179869384, 209.06351348914382, 209.3964542398741, 211.25542058078005, 204.54249749853912, 200.98452348169766, 205.7363863439262, 206.36957013412018, 206.32557016036867, 208.27815402907336, 209.56145513496713, 206.63319895040152, 205.33608639320803, 207.56256709801377, 208.41366157979397, 208.5413906727497, 205.28769469147326, 203.740890192529, 209.38156245307098, 210.122389671683, 205.2721070200644, 206.08194278333343, 209.63072230617183, 209.82289927390238, 208.79120219889498, 208.96165388285027, 208.70713448253298, 209.5264379536008, 2.357475185194937, 1.4639181456914254, 1.9835893762142731, 2.0493370893489584, 1.38241848944349, 2.2821959679097055, 1.7985805894129685, 2.0111785885536575, 1.623401310197683, 1.7538874956851358, 1.5585827017897296, 1.4021457202904453, 1.3899112381927174, 1.2681958896569596, 0.7544763724918945, 1.1647747412107117, 1.5575459896346395, 1.662019265769711, 2.474555301921813, 2.535867596633972, 2.68445533298746, 2.0299961230360375, 2.0790208375025867, 1.6585148384929018, 1.6063681510651249, 2.0666266852569715, 2.160879375242484, 2.1283352831552778, 1.4656326842060838, 1.709762423000586, 1.870584031925758, 2.2082967858355524, 2.2479942572093954, 2.0305464206368815, 2.395759657920259, 2.273092289158334, 1.6503189935778988, 1.885643032416091, 2.038571643986672, 1.7811749557283811, 1.604021199033904, 1.3203166102074078, 0.8829771186669655, 1.3135494660474156, 1.6275240837885119, 0.8636675185318708, 0.9521778200440366, 1.206811016215501, 1.0642462559012336, 0.8301015305970634, 0.8665345507319031, 0.4201279875872637])

8421.662332280257

- La función `F(x)` recibe un vector `x` que contiene las velocidades y los ángulos de ascenso en cada punto de la trayectoria.
- Dentro de la función `F(x)`, se definen varias funciones auxiliares que calculan los valores de las variables de estado en cada punto de la trayectoria, utilizando las ecuaciones de actualización de estado.
- Se define la función `Θ(N_,_��,γ_)`, que calcula las variables de estado (masa, distancia, tiempo, etc.) en cada punto de la trayectoria, teniendo en cuenta las restricciones del problema.
- Se definen funciones para calcular los valores de las variables en el punto final de la trayectoria (después de la fase de ascenso).
- La función `φ(v,γ)` calcula el costo total de la trayectoria, que es una combinación del consumo de combustible y el tiempo de vuelo.

In [ ]:
bounds = [(80, 201.13351856),  (80, 201.13351856),  (80, 201.13351856),  (80, 201.13351856),  (80, 201.13351856),  (80, 211.68630092),  (80, 223.3926229),  (80, 232.02231741),  (80, 236.98506756),  (80, 253.85483296),  (80, 273.13759139),  (80, 274.38892726),  (80, 255.89553494),  (80, 233.61400364),  (80, 237.99529087),  (80, 266.21942107999996),  (80, 255.42662261),  (80, 256.56789835999996),  (80, 256.69206008000003),  (80, 252.86426225),  (80, 258.13899081),  (80, 253.78988898),  (80, 254.20186608),  (80, 256.90219873),  (80, 258.25202577),  (80, 260.29789119),  (80, 258.32651696),  (80, 259.42090009000003),  (80, 256.9320494),  (80, 259.28634189),  (80, 256.23144608999996),  (80, 258.85321431),  (80, 258.33903972999997),  (80, 257.03201203000003),  (80, 260.32304095999996),  (80, 254.03317027),  (80, 256.53821973000004),  (80, 256.44165107000003),  (80, 256.24571273000004),  (80, 257.78129834000003),  (80, 257.18086373),  (80, 256.04700743),  (80, 258.76768403),  (80, 256.17089525),  (80, 255.82555361),  (80, 255.67666213),  (80, 256.95977875),  (80, 256.88906951),  (80, 256.21950953),  (80, 256.59738601000004),  (80, 256.55200501),  (80, 255.97911877),
          (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14), (0, 14)]

population_size = 15

x_ref = np.array([140.4982793587414, 138.93222706627756, 149.16832221025032, 140.5568819970245, 143.19180476170558, 156.9932152847192, 164.75770277319165, 175.75493591692222, 185.29507180047239, 197.34789538855895, 214.0782444255272, 222.30541692351517, 207.86271644148385, 187.42778779939286, 192.00976126288137, 213.5631120192375, 212.49012429333482, 205.02326682419124, 205.54055876062918, 200.20432683743272, 197.7206946785395, 197.62508703628797, 200.33157565347918, 203.58397277885607, 207.49773086234424, 209.19615311890863, 208.78136700291347, 210.0977741852024, 204.96844665971167, 201.5228274316544, 204.7377535487636, 206.58048894510986, 206.50123609377894, 207.40674241720816, 208.38412039202098, 207.4817738500201, 206.68693193109274, 207.53529785421682, 209.4368059637408, 208.34805143301472, 205.60895277700305, 205.88255690007622, 209.18077735789163, 208.9239222798412, 206.6703796304899, 206.8239425724973, 208.58491359242558, 209.46018812482805, 208.5131021419867, 212.28689995559958, 221.24362987224504, 229.66932171520617, 3.0197379365411465, 2.2377882124286907, 1.9684448942467074, 1.9963677215412468, 1.888622917774949, 1.7561056728657018, 1.706897267932226, 1.6360260241290152, 1.6351295777176016, 1.5265078562024577, 1.3089134128536781, 1.2090074620964602, 1.2576370451278795, 1.397727137012875, 1.3433094374755887, 1.2882383989239405, 1.4086269128447666, 1.6823587242896143, 1.7829719200057055, 2.068052983478193, 2.0726459233765753, 2.06796938468013, 2.055487607958022, 1.9529895785267146, 1.9248605344170313, 1.9369190274804877, 1.8800241857702404, 1.9803585133161243, 1.9937772219044394, 1.9583936417466115, 1.9507376711794264, 1.890532501826127, 2.005000469373749, 2.0209651942084896, 1.9719558669737893, 2.0502724437918083, 1.8081149807188954, 1.7360236819604244, 1.612078484679317, 1.517874525119615, 1.3936858369399545, 1.2096235696996014, 1.0598639735425932, 0.9983738206587008, 0.9470253122899331, 0.8044743603462835, 0.7043949153775367, 0.6358197535616844, 0.5355143085341258, 0.4157322148722775, 0.3947659076913997, 0.3878357544699492])

perturbations = [0.1 for _ in range(52)] + [0.01 for _ in range(52)]

print("len (bounds) = ",len(bounds), "     len(x_ref) = ",len(x_ref),"   perturbations = ",len(perturbations) )


atsk = 8014.7

for i in range (2,5000):
    SS = True

    while SS:
        history = []
        def callback(xk, convergence):
            history.append(xk.copy())


        def generate_individual_perturbations(reference_point, perturbations):
            return [ref + np.random.uniform(-perturb, perturb) for ref, perturb in zip(reference_point, perturbations)]


        initial_population = [generate_individual_perturbations(x_ref, perturbations) for _ in range(population_size)]
        result = differential_evolution(FF, bounds, callback=callback, init=np.array(initial_population))

        if result.fun < atsk:
            SS = False

    atsk = result.fun
    print("PUNTO encontrado Angulo:", result.x.tolist())
    print("El valor mínimo es:", result.fun)
    print("i = ",i)
    x_ref = result.x

**Implementación del Algoritmo de Evolución Diferencial**

- Se define la función `objective_function(x)`, que se utiliza como la función objetivo para el algoritmo de Evolución Diferencial.
- Se define la función `FF(x)`, que es una versión modificada de la función objetivo que se utiliza para manejar los casos en los que la función `F(x)` retorna valores no numéricos (NaN).
- Se definen los límites de las variables de optimización en la lista `bounds`.
- Se define un punto de referencia inicial `x_ref`.
- Se define un vector de perturbaciones `perturbations` que se utiliza para generar la población inicial del algoritmo de Evolución Diferencial.
- Se define el tamaño de la población `population_size`.
- Se implementa un bucle que ejecuta el algoritmo de Evolución Diferencial varias veces, utilizando el punto de referencia anterior como punto de partida para la siguiente iteración.
- Se utiliza la función `differential_evolution` de la biblioteca SciPy para implementar el algoritmo de Evolución Diferencial.
- Se define una función `callback` que se utiliza para registrar el historial de la optimización.
- Se define una función `generate_individual_perturbations` que se utiliza para generar la población inicial del algoritmo de Evolución Diferencial, utilizando el punto de referencia y el vector de perturbaciones.
- Se imprimen los resultados de la optimización (el punto óptimo encontrado y el valor mínimo de la función objetivo).

El código implementa un algoritmo de Evolución Diferencial para optimizar la trayectoria de ascenso de una aeronave A320, minimizando una función de costo que considera el consumo de combustible y el tiempo de vuelo. La implementación incluye funciones auxiliares para el cálculo de variables aerodinámicas y atmosféricas, así como un mecanismo de perturbación para explorar el espacio de soluciones de forma más eficiente. El resultado de la optimización proporcionaría la combinación óptima de velocidades y ángulos de ascenso en cada punto de la trayectoria discretizada, lo que permite minimizar el costo total del vuelo.